#*************Parameters***************************

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
%sql
SELECT * FROM workspace.silver.silver_passengers

In [0]:
%sql
select * from workspace.silver.silver_passengers where passenger_id='P0049'

In [0]:
%sql
SELECT * FROM workspace.gold.dimpassengers WHERE passenger_id='P0049'

In [0]:
key_cols = "['passenger_id']"
key_cols_list = eval(key_cols)
catalog = "workspace"


In [0]:
key_cols

In [0]:
key_cols_list

###CDC column

In [0]:
cdc_col = "modified_date"

###Backdated Refresh

In [0]:
backdated_refresh = ""

###Source Object

In [0]:
source_object = "silver_passengers"

###Source Schema 

In [0]:
source_schema = "silver"

#Target Schema

In [0]:
target_schema = "gold"

In [0]:
target_object = "DimPassengers"

In [0]:
surrogate_key = "DimPassengersKey"

#Last Load Date

In [0]:
if len(backdated_refresh)==0:
    if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
        last_load = spark.sql(f"select max({cdc_col}) from {catalog}.{target_schema}.{target_object}").collect()[0][0]
    else:
        last_load = "1900-01-01 00:00:00"
else:
    last_load = backdated_refresh

        

In [0]:
last_load

In [0]:
#Testing
df_src = spark.sql(f"select * from {source_schema}.{source_object} where {cdc_col}>'{last_load}'")

In [0]:
df_src.display()

#Old VS NEW

In [0]:
key_cols_list

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    key_cols_string_incremental = ", ".join(key_cols_list)

    df_trg = spark.sql(f"""select {key_cols_string_incremental}, {surrogate_key}, create_date, update_date from {catalog}.{target_schema}.{target_object}""")

else:
    #key columns string for Initial
    key_cols_string_init = [f"'' AS {i}" for i in key_cols_list]
    key_cols_string_init = ", ".join(key_cols_string_init)
    df_trg = spark.sql(f"""select {key_cols_string_init}, CAST('0' AS INT) AS {surrogate_key},CAST('1900-01-01 00:00:00' AS timestamp) AS create_date,CAST('1900-01-01 00:00:00' AS timestamp) AS update_date WHERE 1=0""")


In [0]:
df_trg.display()

#Join Condition

In [0]:
join_condition = ' AND '.join([f"src.{i} = trg.{i}" for i in key_cols_list])

In [0]:
join_condition

In [0]:
df_src.display()

In [0]:
df_trg.display()

In [0]:
df_src.createOrReplaceTempView("src")
df_trg.createOrReplaceTempView("trg")

df_join = spark.sql(f"""SELECT src.*, trg.{surrogate_key}, trg.create_date, trg.update_date FROM src LEFT JOIN trg ON {join_condition}""")

In [0]:
df_join.display()

#Old REcords

In [0]:
df_old = df_join.filter(col(f"{surrogate_key}").isNotNull())
df_old.display()
df_new = df_join.filter(col(f"{surrogate_key}").isNull())
df_new.display()
df_old.count()

#Preparing the DF_OLD

In [0]:
df_old_enr = df_old.withColumn("update_date", current_timestamp())

In [0]:
df_old_enr.display()

# Preparing the DF_NEW

In [0]:
df_new.display()

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    max_surrogate_key = spark.sql(f"""select max({surrogate_key}) from {catalog}.{target_schema}.{target_object}""").collect()[0][0]

    df_new_enr = df_new.withColumn(f'{surrogate_key}', lit(max_surrogate_key) + lit(1)+ monotonically_increasing_id())\
                       .withColumn('create_date', current_timestamp())\
                       .withColumn('update_date', current_timestamp())
else:
    max_surrogate_key = 0
    df_new_enr = df_new.withColumn(f'{surrogate_key}', lit(max_surrogate_key) + lit(1)+ monotonically_increasing_id())\
                        .withColumn('create_date', current_timestamp())\
                        .withColumn('update_date', current_timestamp())
df_new_enr.display()

In [0]:
df_old_enr.display()

#Union old and New records

In [0]:
df_union = df_old_enr.unionByName(df_new_enr)

In [0]:
df_union.display()

###Modified_date column was the original column to compare target_object "Dim_flights"

In [0]:
surrogate_key

In [0]:
if spark.catalog.tableExists(f"{catalog}.{target_schema}.{target_object}"):
    dlt_obj = DeltaTable.forName(spark, f"{catalog}.{target_schema}.{target_object}")
    dlt_obj.alias("trg").merge(df_union.alias("src"),
        f"trg.{surrogate_key} = src.{surrogate_key}")\
        .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
        .execute()
else:
    df_union.write.format("delta")\
        .mode("append")\
        .saveAsTable(f"{catalog}.{target_schema}.{target_object}")

#Validation in the Gold Layer

In [0]:
%sql
SELECT * FROM workspace.gold.dimpassengers WHERE passenger_id='P0049'

In [0]:
%sql
SELECT * FROM workspace.silver.silver_passengers WHERE passenger_id='P0049'

In [0]:
%sql
select * from workspace.gold.dimpassengers

In [0]:
%sql
select * from workspace.silver.silver_passengers

In [0]:
df_gold = spark.sql(f"select * from {catalog}.{target_schema}.{target_object}")

df_gold.display()

In [0]:
spark.sql(f"DELETE FROM {catalog}.{target_schema}.{target_object} WHERE passenger_id > 'P0220'")

In [0]:
df_gold_df = spark.read.format("csv")\
            .option("inferSchema",True)\
            .option('header', True)\
            .load("workspace.gold.dimairports")

In [0]:
df_gold_df.display()